In [ ]:
# recalc_opts.py
def recalc_opts(ib, df):
    '''Recalculates option prices and margins
    Arg:
       (ib) as connection object
       (df) as the dataframe of options
    Returns: dataframe with updated OptPrices and OptMargins
    '''
    opt_contracts = [Contract(conId=c) for c in df.optId]
    qopts = ib.qualifyContracts(*opt_contracts)
    tickers = ib.reqTickers(*qopts)
    optPrices = {t.contract.conId: t.marketPrice() for t in tickers} # {symbol: undPrice}
    optMargins = {c.conId: ib.whatIfOrder(c,Order(action='SELL', orderType='MKT', totalQuantity=abs(ls), whatIf=True)).initMarginChange 
        for c, ls in (zip(qopts, df.lotsize))}
    
    # update prices and margins
    df = df.set_index('optId')
    df.optPrice.update(pd.Series(optPrices))
    df.optMargin.update(pd.Series(optMargins))
    df.optMargin = df.optMargin.astype('float')
    df = df.assign(rom=df.optPrice*df.lotsize/df.optMargin*252/df.dte)
    
    return df.reset_index()

In [ ]:
import pandas as pd

from os import listdir
from ib_insync import *
util.startLoop()

from helper import get_nse_remqty, get_connected, grp_opts, get_prec, recalc_opts

fspath = '../data/nse/'  # path for nse pickles

minOptPrice = 0.15
minRom = 0.25
nLargest = 5

fs = listdir(fspath)

# get the remaining quantities
with get_connected('nse', 'live') as ib:
    remqty = get_nse_remqty(ib)

optsList = [f for f in fs if f[-3:] == 'pkl']

cols = ['optId', 'symbol', 'right', 'expiration', 'dte', 'strike', 'undPrice', 
'lo52', 'hi52', 'Fall', 'Rise', 'loFall', 'hiRise', 'std3', 'loStd3', 'hiStd3', 
'lotsize', 'optPrice', 'optMargin', 'rom']

df1 = pd.concat([pd.read_pickle(fspath+f) 
                 for f in optsList], axis=0, sort=True).reset_index(drop=True)[cols]

# filter for high probability
df2 = df1[((df1.strike > df1.hi52) | 
           (df1.strike < df1.lo52)) & 
          (df1.optPrice > minOptPrice) & 
          (df1.rom > minRom)]

df2 = df2.sort_values('rom', ascending=False)
df2[df2.rom > minRom].reset_index(drop=True)

In [5]:
# recalculated the prices and margins
with get_connected('nse', 'live') as ib:
    df2 = recalc_opts(ib, df2)

Started to throttle requests
Error 0, reqId -1: Warning: Approaching max rate of 50 messages per second (44)
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests


In [25]:
# group by the largest N return on margin
df2 = grp_opts(df2)
df3 = df2.groupby('symbol').apply(lambda x: x.nlargest(nLargest, 'rom'))

df4 = df3.assign(expPrice=[get_prec(p*1.1, 0.05) for p in df3.optPrice])
df4 = df4.assign(remqty=[remqty[u] for u in df4.symbol])
df4 = df4.assign(expQty=df4.remqty)

In [29]:
df4 = df4[df4.remqty != 0] # remove zero remaining quantities (blacklisted)

In [43]:
df4

optId     symbol right expiration  dte  strike  undPrice      lo52      hi52    Fall     Rise   loFall    hiRise        std3       loStd3       hiStd3  lotsize  optPrice   optMargin       rom  expPrice  remqty  expQty
symbol                                                                                                                                                                                                                                      
ACC       59   355720294        ACC     C   20190425    6  1860.0   1703.00  1256.000  1857.450  220.95  283.800  1482.05  1986.800  107.978226  1595.021774  1810.978226      400      1.05   123964.91  0.142298      1.15       2       2
ADANIENT  60   359177372   ADANIENT     C   20190425    6   180.0    143.05    72.025   179.500   33.10   49.250   109.95   192.300   14.247848   128.802152   157.297848     4000      0.25   207227.83  0.202675      0.30       2       2
          61   359177381   ADANIENT     C   20190425    6   185.0    143.05    72.025   179.500   33.10   49.250   109.95   192.300   14.247848   128.802152   157.297848     4000      0.25   207227.83  0.202675      0.30       2       2
          62   359177396   ADANIENT     C   20190425    6   195.0    143.05    72.025   179.500   33.10   49.250   109.95   192.300   14.247848   128.802152   157.297848     4000      0.25   207227.83  0.202675      0.30       2       2
          63   359177406   ADANIENT     C   20190425    6   200.0    143.05    72.025   179.500   33.10   49.250   109.95   192.300   14.247848   128.802152   157.297848     4000      0.20   207227.83  0.162140      0.20       2       2
ADANIPORT 64   355582726  ADANIPORT     C   20190530   41   470.0    395.65   294.100   452.200  112.20  107.700   283.45   503.350   70.545227   325.104773   466.195227     2500      2.15   179800.08  0.183741      2.35       1       1
          65   355582732  ADANIPORT     C   20190530   41   480.0    395.65   294.100   452.200  112.20  107.700   283.45   503.350   70.545227   325.104773   466.195227     2500      1.45   179800.08  0.123918      1.60       1       1
ADANIPOWE 66   352120658  ADANIPOWE     C   20190425    6    65.0     54.35    15.150    58.350   13.55   20.300    40.80    74.650    7.363739    46.986261    61.713739    20000      0.20   463671.13  0.362326      0.20       1       1
AMARAJABA 1    359168387  AMARAJABA     P   20190425    6   600.0    706.25   650.000   908.450  130.30  151.500   575.95   857.750   54.412305   651.837695   760.662305      700      0.65    88308.85  0.216400      0.70       3       3
          2    359168864  AMARAJABA     P   20190425    6   590.0    706.25   650.000   908.450  130.30  151.500   575.95   857.750   54.412305   651.837695   760.662305      700      0.65    88308.85  0.216400      0.70       3       3
          4    359168857  AMARAJABA     P   20190425    6   570.0    706.25   650.000   908.450  130.30  151.500   575.95   857.750   54.412305   651.837695   760.662305      700      0.40    88308.85  0.133169      0.45       3       3
          0    359169035  AMARAJABA     P   20190530   41   610.0    706.25   650.000   908.450  204.40  197.450   501.85   903.700   89.129700   617.120300   795.379700      700      2.50    88308.85  0.121801      2.75       3       3
          5    359168371  AMARAJABA     P   20190425    6   560.0    706.25   650.000   908.450  130.30  151.500   575.95   857.750   54.412305   651.837695   760.662305      700      0.35    88308.85  0.116523      0.40       3       3
APOLLOHOS 67   359208741  APOLLOHOS     C   20190530   41  1520.0   1275.20   910.100  1387.200  304.10  324.050   971.10  1599.250  225.697854  1049.502146  1500.897854      500      6.10   116162.32  0.161381      6.70       2       2
          68   359208751  APOLLOHOS     C   20190530   41  1540.0   1275.20   910.100  1387.200  304.10  324.050   971.10  1599.250  225.697854  1049.502146  1500.897854      500      4.80   116162.32  0.126988      5.30       

In [ ]:
# make calls and puts watch - to quickly weed out risky options
gb = df5.groupby('right')

if 'C' in [k for k in gb.indices]:
#     df5_calls = gb.get_group('C').reset_index(drop=True).sort_values(['symbol', 'strike'], ascending=[True, True])
    df5_calls = gb.get_group('C').reset_index(drop=True)
    
    df5_callsymbols = df5_calls.symbol.unique()
    watchcalls = [('DES', s, 'STK', 'NSE') if s not in ['NIFTY50', 'BANKNIFTY'] else ('DES', s, 'IND', 'NSE') for s in df5_callsymbols]
    df5_wp = util.df(watchcalls)
    df5_wp.to_csv(fspath+'callswatch.csv', index=None, header=False)
    
else:
    df5_calls = pd.DataFrame([]) # empty dataframe

if 'P' in [k for k in gb.indices]:
#     df5_puts = gb.get_group('P').reset_index(drop=True).sort_values(['symbol', 'strike'], ascending=[True, False])
    df5_puts = gb.get_group('P').reset_index(drop=True)
    
    # make watchlist
    df5_putssymbols = df5_puts.symbol.unique()
    watchputs = [('DES', s, 'STK', 'NSE') if s not in ['NIFTY50', 'BANKNIFTY'] else ('DES', s, 'IND', 'NSE') for s in df5_putssymbols]
    df5_wp = util.df(watchputs)
    df5_wp.to_csv(fspath+'putswatch.csv', index=None, header=False)
else:
    df5_puts = pd.DataFrame([]) # empty dataframe

# output the consolidated puts and calls dataframe
df5 = pd.concat([df5_puts, df5_calls]).reset_index(drop=True)
df5.to_csv(fspath+'check.csv', index=None, header=True)

In [ ]:
df5 = df5.assign(expQty=3)

In [ ]:
df5.loc[df5.symbol == 'BSOFT', 'expPrice'] = 0.55

In [ ]:
df5

In [ ]:
sum(df5.expQty*df5.expPrice*df5.lotsize)

In [ ]:
opt_contracts = [Contract(conId=c) for c in df5.optId]
with get_connected('nse', 'live') as ib:
    qopts = ib.qualifyContracts(*opt_contracts)

In [ ]:
orders = [LimitOrder('SELL', qty*lotsize, price) for qty, lotsize, price in zip(df5.expQty, df5.lotsize, df5.expPrice)]

In [ ]:
with get_connected('nse', 'live') as ib:
    limitTrade = [ib.placeOrder(c, o) for c, o in zip(qopts,  orders)]

In [ ]:
# ib = get_connected('nse', 'live')

In [ ]:
[ib.placeOrder(c, o) for c, o in zip(qopts,  orders)]

In [ ]:
ib.reqGlobalCancel()

In [ ]:
import pandas as pd
df_temp = pd.read_pickle('../data/nse/BHARATFIN.pkl')

In [ ]:
df_temp

In [ ]:
df_temp.loc[:, 'lotsize'] = 500

df_temp.to_pickle('../data/nse/BHARATFIN.pkl')